# Introduction
---
In this notebook I am developing methods for MCMC sampling.

In [4]:
import sys
sys.path.append('../')
from sampler import MCMC

In [5]:
optimizer = MCMC.SequenceOptimizer(native_seq='AHVFGRACY')   
print(optimizer)

ProteusAI.MCMC.SequenceOptimizer class: 
---------------------------------------
When SequenceOptimizer.run() is called sequence optimization will be performed on sequence:

AHVFGRACY

The following variables were set:

variable	|value
----------------+-------------------
sampler: 	|simulated_annealing
n_traj: 	|5
n_iter: 	|1000
mut_p: 		|(0.6, 0.2, 0.2)
T: 		|10.0
M: 		|0.01

The energy function is a linear combination of the following constraints:

constraint	|value	|weight
----------------+-------+------------
length 		|200	|0.0



In [3]:
optimizer.mutate

/Users/jonathanfunk/Projects/GitHub/proteusAI/notebooks/../sampler/MCMC.py:133: RuntimeWarning: divide by zero encountered in divide
  exp_val = np.exp(1 / (T * dE))


['LYSQYENIKVVAQTCRN', 'IRVAINDAQ', 'MSA', 'YRQ', 'SNHCWVI']

In [40]:
import numpy as np

pdb = []
with open('../scripts/accepted/sequence_0_iter_101.pdb', 'r') as f:
    for line in f:
        pdb.append(line)
pdb = ''.join(pdb)


def globularity(pdbs: str):
    """
    globularity constraint
    
    Parameters:
        pdb (list): list of pdb_strings
    
    Returns:
        np.array: variances
    """
    variances = np.zeros(len(pdbs))

    for i, pdb in enumerate(pdbs):
        coordinates = []
        pdb = pdb.split('\n')
        for line in pdb:
            if line.startswith('ATOM'):
                x = float(line[32:38])
                y = float(line[39:46])
                z = float(line[46:54])
                coordinates.append(np.array([x, y, z]))
                

        variance = sum(np.var(coordinates, axis=0, ddof=1))
        variances[i] = variance.item()

    return variances
            

globularity([pdb, pdb])*0.002

array([0.25497309, 0.25497309])

In [13]:
len('ATOM      1  N   THR A   1       ')

33